In [38]:
import pandas as pd

In [39]:
df_wahl = pd.read_csv(r'C:\Users\DE88342\OneDrive - Grunenthal Group\Desktop\VSC - Py\DPArenamed\Data\IMPORT BOT2_Wahl.csv', delimiter= ';')
df_event = pd.read_csv(r'C:\Users\DE88342\OneDrive - Grunenthal Group\Desktop\VSC - Py\DPArenamed\Data\IMPORT BOT1_Veranstaltungsliste.csv', delimiter= ';')
df_raum = pd.read_csv(r'C:\Users\DE88342\OneDrive - Grunenthal Group\Desktop\VSC - Py\DPArenamed\Data\IMPORT BOT0_Raumliste.csv', delimiter= ';')

In [40]:
df_wahl.head(20)

,Klasse,Name,Vorname,Wahl 1,Wahl 2,Wahl 3,Wahl 4,Wahl 5,Wahl 6
0,ASS221,Bigay,Ali Eren,6.0,18.0,19.0,4.0,14.0,22.0
1,ASS221,Graf,Lena,16.0,17.0,9.0,18.0,19.0,15.0
2,ASS221,Görgen,Smilla Marie,20.0,25.0,1.0,15.0,26.0,11.0
3,ASS221,Iskender,Iclal Ece,10.0,2.0,6.0,16.0,18.0,15.0
4,ASS221,Karakas,Berfin,2.0,26.0,13.0,15.0,22.0,23.0
5,ASS221,Kavak,Hazal,NaN,NaN,NaN,NaN,NaN,NaN
6,ASS221,Martins,Ayo,6.0,16.0,18.0,23.0,24.0,25.0
7,ASS221,Mohammed,Zakariah,7.0,13.0,14.0,16.0,18.0,26.0
8,ASS221,Nguyen,Hai Nam,3.0,6.0,14.0,26.0,19.0,22.0
9,ASS221,Ntumba Tshikala,Ruth,3.0,18.0,23.0,4.0,15.0,16.0


In [47]:
value_counts = pd.concat([df_wahl['Wahl 1'].value_counts(),
                         df_wahl['Wahl 2'].value_counts(),
                         df_wahl['Wahl 3'].value_counts()])

df_value_counts = pd.DataFrame(value_counts)
print(df_value_counts)


      count
19.0     22
18.0     11
10.0     10
26.0     10
9.0       9
...     ...
27.0      2
25.0      2
5.0       1
22.0      1
2.0       1

[77 rows x 1 columns]


In [45]:

value_counts['Sum'] = value_counts.sum()
value_counts.head()


19.0    22
18.0    11
10.0    10
26.0    10
9.0      9
Name: count, dtype: int64

In [ ]:
value_counts.describe()

,count,count,count,count,count,count,Sum
count,24.000000,26.000000,27.000000,25.000000,26.000000,26.000000,27.000000
mean,5.541667,5.115385,4.925926,5.320000,5.076923,5.038462,29.444444
std,4.827270,4.942282,3.395992,2.868217,2.225724,2.630297,15.457966
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000
25%,2.000000,2.000000,3.000000,3.000000,4.000000,2.250000,17.500000
50%,4.000000,4.000000,4.000000,5.000000,4.500000,5.500000,26.000000
75%,8.250000,5.750000,5.500000,7.000000,7.000000,7.000000,41.000000
max,22.000000,20.000000,14.000000,10.000000,10.000000,9.000000,68.000000
